### Generate plots

Before running this notebook you have to generate the input dataframe, **df_all.tsv**, with the main.py script.
Optionally, a mapping file specifying group and label of methods can be used to select only one method (the best) for each group and each namespace.

#### Output
* 3 figures, *fig_<metric>_<namespace>.png*
* 1 TSV, *eval_<metric>.tsv*

In [60]:
# Input
df_file = "../../CAFA-evaluator_data/CAFA3/results/df_all.tsv"
out_folder = "../../CAFA-evaluator_data/CAFA3/results"

# Set to None if you don't want to use it. Results will not be grouped
# name_file = None
names_file = "../../CAFA-evaluator_data/CAFA3/names.tsv"

# Select a metric
# metric, cols = ('f', ['rc', 'pr'])
metric, cols =  ('wf', ['wrc', 'wpr'])
# metric, cols = ('s', ['ru', 'mi'])

In [61]:
import pandas as pd
import matplotlib.pyplot as plt
plt.set_loglevel("info")

In [62]:
df = pd.read_csv(df_file, sep="\t")
df

,filename,ns,tau,cov,pr,rc,f,wpr,wrc,wf,mi,ru,s
0,ZhuLab1_GOLabeler-2015.cafa,biological_process,0.01,1.000000,0.020549,0.845575,0.040122,0.014715,0.747700,0.028862,5609.935844,33.454652,5610.035596
1,ZhuLab1_GOLabeler-2015.cafa,biological_process,0.02,1.000000,0.020549,0.845575,0.040122,0.014715,0.747700,0.028862,5609.935844,33.454652,5610.035596
2,ZhuLab1_GOLabeler-2015.cafa,biological_process,0.03,1.000000,0.020549,0.845575,0.040122,0.014715,0.747700,0.028862,5609.935844,33.454652,5610.035596
3,ZhuLab1_GOLabeler-2015.cafa,biological_process,0.04,1.000000,0.020549,0.845575,0.040124,0.014715,0.747700,0.028862,5609.674815,33.454652,5609.774572
4,ZhuLab1_GOLabeler-2015.cafa,biological_process,0.05,1.000000,0.020554,0.845575,0.040132,0.014719,0.747700,0.028870,5608.283352,33.454652,5608.383133
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11989,sysbiogroup.cafa,molecular_function,0.95,0.716186,0.487609,0.368449,0.419735,0.463754,0.338457,0.391321,41.472390,41.550152,58.705828
11990,sysbiogroup.cafa,molecular_function,0.96,0.716186,0.487609,0.368449,0.419735,0.463754,0.338457,0.391321,41.472390,41.550152,58.705828
11991,sysbiogroup.cafa,molecular_function,0.97,0.716186,0.487609,0.368449,0.419735,0.463754,0.338457,0.391321,41.472390,41.550152,58.705828
11992,sysbiogroup.cafa,molecular_function,0.98,0.716186,0.487609,0.368449,0.419735,0.463754,0.338457,0.391321,41.472390,41.550152,58.705828


In [63]:
methods = pd.read_csv(names_file, delim_whitespace=True)
methods

,filename,group,label
0,anacletolab.cafa,anacletolab,anacletolab
1,caolab1.cafa,caolab,caolab1
2,caolab4_1.cafa,caolab,caolab4_1
3,caolab4_2.cafa,caolab,caolab4_2
4,caolab4_3.cafa,caolab,caolab4_3
5,cbrcborg_1.cafa,cbrcborg,cbrcborg_1
6,cbrcborg_2.cafa,cbrcborg,cbrcborg_2
7,cbrcborg_3.cafa,cbrcborg,cbrcborg_3
8,DeepGO_1.cafa,DeepGO,DeepGO_1
9,DeepGO_2.cafa,DeepGO,DeepGO_2


In [64]:
if names_file is None:
    df['group'] = df['filename']
    df['label'] = df['filename']
    df['is_baseline'] = False
else:
    # Set method information (optional)
    df = pd.merge(df, methods, on='filename', how='left')
    df['group'].fillna(df['filename'], inplace=True)
    df['label'].fillna(df['filename'], inplace=True)
    if 'is_baseline' not in df:
        df['is_baseline'] = False
    else:
        df['is_baseline'].fillna(False, inplace=True)
df = df.drop(columns='filename').set_index(['group', 'label', 'ns', 'tau'])
df

cov  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  1.000000   
                                                      0.02  1.000000   
                                                      0.03  1.000000   
                                                      0.04  1.000000   
                                                      0.05  1.000000   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.716186   
                                                      0.96  0.716186   
                                                      0.97  0.716186   
                                                      0.98  0.716186   
                                                      0.99  0.716186   

                                                                  pr  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.020549   
                                                      0.02  0.020549   
                                                      0.03  0.020549   
                                                      0.04  0.020549   
                                                      0.05  0.020554   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.487609   
                                                      0.96  0.487609   
                                                      0.97  0.487609   
                                                      0.98  0.487609   
                                                      0.99  0.487609   

                                                                  rc  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.845575   
                                                      0.02  0.845575   
                                                      0.03  0.845575   
                                                      0.04  0.845575   
                                                      0.05  0.845575   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.368449   
                                                      0.96  0.368449   
                                                      0.97  0.368449   
                                                      0.98  0.368449   
                                                      0.99  0.368449   

                                                                   f  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.040122   
                                                      0.02  0.040122   
                                                      0.03  0.040122   
                                                      0.04  0.040124   
                                                      0.05  0.040132   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.419735   
                                                      0.96  0.419735   
                                                      0.97  0.419735   
                                                      0.98  0.419735   
                                                      0.99  0.419735   

                                                                 wpr  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.014715   
                                                      0.02  0.014715   
                             

In [65]:
# Assign colors based on group
cmap = plt.get_cmap('tab20')
df['colors'] = df.index.get_level_values('group')
df['colors'] = pd.factorize(df['colors'])[0]
df['colors'] = df['colors'].apply(lambda x: cmap.colors[x % len(cmap.colors)])
df

cov  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  1.000000   
                                                      0.02  1.000000   
                                                      0.03  1.000000   
                                                      0.04  1.000000   
                                                      0.05  1.000000   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.716186   
                                                      0.96  0.716186   
                                                      0.97  0.716186   
                                                      0.98  0.716186   
                                                      0.99  0.716186   

                                                                  pr  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.020549   
                                                      0.02  0.020549   
                                                      0.03  0.020549   
                                                      0.04  0.020549   
                                                      0.05  0.020554   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.487609   
                                                      0.96  0.487609   
                                                      0.97  0.487609   
                                                      0.98  0.487609   
                                                      0.99  0.487609   

                                                                  rc  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.845575   
                                                      0.02  0.845575   
                                                      0.03  0.845575   
                                                      0.04  0.845575   
                                                      0.05  0.845575   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.368449   
                                                      0.96  0.368449   
                                                      0.97  0.368449   
                                                      0.98  0.368449   
                                                      0.99  0.368449   

                                                                   f  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.040122   
                                                      0.02  0.040122   
                                                      0.03  0.040122   
                                                      0.04  0.040124   
                                                      0.05  0.040132   
...                                                              ...   
sysbiogroup sysbiogroup            molecular_function 0.95  0.419735   
                                                      0.96  0.419735   
                                                      0.97  0.419735   
                                                      0.98  0.419735   
                                                      0.99  0.419735   

                                                                 wpr  \
group       label                  ns                 tau              
ZhuLab1     ZhuLab1_GOLabeler-2015 biological_process 0.01  0.014715   
                                                      0.02  0.014715   
                             

In [66]:
# Identify best methods and thresholds
index_best = df.groupby(level=['group', 'ns'])[metric].idxmax() if metric in ['f', 'wf'] else df.groupby(['group', 'ns'])[metric].idxmin()
index_best

group             ns                
DeepGO            biological_process         (DeepGO, DeepGO_1, biological_process, 0.09)
                  cellular_component    (DeepGO, DeepGO_1, cellular_component, 0.09999...
                  molecular_function         (DeepGO, DeepGO_2, molecular_function, 0.41)
INGA              biological_process             (INGA, INGA_1, biological_process, 0.44)
                  cellular_component             (INGA, INGA_1, cellular_component, 0.55)
                                                              ...                        
tianlab1          cellular_component       (tianlab1, tianlab1, cellular_component, 0.39)
                  molecular_function       (tianlab1, tianlab1, molecular_function, 0.35)
zhang_freddolino  biological_process    (zhang_freddolino, zhang_freddolino_1, biologi...
                  cellular_component    (zhang_freddolino, zhang_freddolino_2, cellula...
                  molecular_function    (zhang_freddolino, zhan

In [67]:
# Filter the dataframe for the best method and threshold
df_best = df.loc[index_best, ['cov', 'colors'] + cols + [metric]]
df_best

cov  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.09  0.851064   
                                    cellular_component 0.10  0.800940   
                 DeepGO_2           molecular_function 0.41  0.873614   
INGA             INGA_1             biological_process 0.44  0.930260   
                                    cellular_component 0.55  0.968652   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.39  0.888715   
                                    molecular_function 0.35  0.849224   
zhang_freddolino zhang_freddolino_1 biological_process 0.41  0.992908   
                 zhang_freddolino_2 cellular_component 0.39  0.995298   
                 zhang_freddolino_1 molecular_function 0.64  0.815965   

                                                                                                        colors  \
group            label              ns                 tau                                                       
DeepGO           DeepGO_1           biological_process 0.09  (0.09019607843137255, 0.7450980392156863, 0.81...   
                                    cellular_component 0.10  (0.09019607843137255, 0.7450980392156863, 0.81...   
                 DeepGO_2           molecular_function 0.41  (0.09019607843137255, 0.7450980392156863, 0.81...   
INGA             INGA_1             biological_process 0.44  (0.8392156862745098, 0.15294117647058825, 0.15...   
                                    cellular_component 0.55  (0.8392156862745098, 0.15294117647058825, 0.15...   
...                                                                                                        ...   
tianlab1         tianlab1           cellular_component 0.39  (0.7372549019607844, 0.7411764705882353, 0.133...   
                                    molecular_function 0.35  (0.7372549019607844, 0.7411764705882353, 0.133...   
zhang_freddolino zhang_freddolino_1 biological_process 0.41  (0.596078431372549, 0.8745098039215686, 0.5411...   
                 zhang_freddolino_2 cellular_component 0.39  (0.596078431372549, 0.8745098039215686, 0.5411...   
                 zhang_freddolino_1 molecular_function 0.64  (0.596078431372549, 0.8745098039215686, 0.5411...   

                                                                  wrc  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.09  0.120647   
                                    cellular_component 0.10  0.201405   
                 DeepGO_2           molecular_function 0.41  0.135108   
INGA             INGA_1             biological_process 0.44  0.346230   
                                    cellular_component 0.55  0.569174   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.39  0.480395   
                                    molecular_function 0.35  0.384151   
zhang_freddolino zhang_freddolino_1 biological_process 0.41  0.283666   
                 zhang_freddolino_2 cellular_component 0.39  0.497686   
                 zhang_freddolino_1 molecular_function 0.64  0.352209   

                                                                  wpr  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.09  0.124663   
                                    cellular_component 0.10  0.251971   
                 DeepGO_2           molecular_function 0.41  0.137387   
INGA             INGA_1             biological_process 0.44  0.305054   
                                    cellular_component 0.55  0.449868   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.39  0.449465   
                          

In [68]:
# Filter the dataframe for the best methods
df_methods = df.reset_index('tau').loc[[ele[:-1] for ele in index_best], ['tau', 'cov', 'colors'] + cols + [metric]].sort_index()
# Makes the curves monotonic. Cumulative max on the last column of the cols variable, e.g. "pr" --> precision
df_methods[cols[-1]] = df_methods.groupby(level=['label', 'ns'])[cols[-1]].cummax()
# Save to file
df_methods.drop(columns=['colors']).to_csv('{}/eval_{}.tsv'.format(out_folder, metric), float_format="%.3f", sep="\t")
df_methods

tau       cov  \
group            label              ns                                   
DeepGO           DeepGO_1           biological_process  0.01  0.934988   
                                    biological_process  0.02  0.934988   
                                    biological_process  0.03  0.934988   
                                    biological_process  0.04  0.934988   
                                    biological_process  0.05  0.934988   
...                                                      ...       ...   
zhang_freddolino zhang_freddolino_2 cellular_component  0.95  0.824451   
                                    cellular_component  0.96  0.807210   
                                    cellular_component  0.97  0.807210   
                                    cellular_component  0.98  0.763323   
                                    cellular_component  0.99  0.694357   

                                                                                                   colors  \
group            label              ns                                                                      
DeepGO           DeepGO_1           biological_process  (0.09019607843137255, 0.7450980392156863, 0.81...   
                                    biological_process  (0.09019607843137255, 0.7450980392156863, 0.81...   
                                    biological_process  (0.09019607843137255, 0.7450980392156863, 0.81...   
                                    biological_process  (0.09019607843137255, 0.7450980392156863, 0.81...   
                                    biological_process  (0.09019607843137255, 0.7450980392156863, 0.81...   
...                                                                                                   ...   
zhang_freddolino zhang_freddolino_2 cellular_component  (0.596078431372549, 0.8745098039215686, 0.5411...   
                                    cellular_component  (0.596078431372549, 0.8745098039215686, 0.5411...   
                                    cellular_component  (0.596078431372549, 0.8745098039215686, 0.5411...   
                                    cellular_component  (0.596078431372549, 0.8745098039215686, 0.5411...   
                                    cellular_component  (0.596078431372549, 0.8745098039215686, 0.5411...   

                                                             wrc       wpr  \
group            label              ns                                       
DeepGO           DeepGO_1           biological_process  0.156530  0.092760   
                                    biological_process  0.156530  0.092760   
                                    biological_process  0.156530  0.092760   
                                    biological_process  0.156530  0.092760   
                                    biological_process  0.156530  0.092760   
...                                                          ...       ...   
zhang_freddolino zhang_freddolino_2 cellular_component  0.166574  0.787069   
                                    cellular_component  0.155077  0.799912   
                                    cellular_component  0.155077  0.799912   
                                    cellular_component  0.142141  0.815328   
                                    cellular_component  0.127364  0.829649   

                                                              wf  
group            label              ns                            
DeepGO           DeepGO_1           biological_process  0.116489  
                                    biological_process  0.116489  
                                    biological_process  0.116489  
                                    biological_process  0.116489  
                                    biological_process  0.116489  
...                                                          ...  
zhang_freddolino zhang_freddolino_2 cellular_component  0.274957  
                                    cellular_component  0.

In [69]:
# Calculate the max coverage across all thresholds
df_best['max_cov'] = df_methods.groupby(level=['group', 'label', 'ns'])['cov'].max()
df_best

cov  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.09  0.851064   
                                    cellular_component 0.10  0.800940   
                 DeepGO_2           molecular_function 0.41  0.873614   
INGA             INGA_1             biological_process 0.44  0.930260   
                                    cellular_component 0.55  0.968652   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.39  0.888715   
                                    molecular_function 0.35  0.849224   
zhang_freddolino zhang_freddolino_1 biological_process 0.41  0.992908   
                 zhang_freddolino_2 cellular_component 0.39  0.995298   
                 zhang_freddolino_1 molecular_function 0.64  0.815965   

                                                                                                        colors  \
group            label              ns                 tau                                                       
DeepGO           DeepGO_1           biological_process 0.09  (0.09019607843137255, 0.7450980392156863, 0.81...   
                                    cellular_component 0.10  (0.09019607843137255, 0.7450980392156863, 0.81...   
                 DeepGO_2           molecular_function 0.41  (0.09019607843137255, 0.7450980392156863, 0.81...   
INGA             INGA_1             biological_process 0.44  (0.8392156862745098, 0.15294117647058825, 0.15...   
                                    cellular_component 0.55  (0.8392156862745098, 0.15294117647058825, 0.15...   
...                                                                                                        ...   
tianlab1         tianlab1           cellular_component 0.39  (0.7372549019607844, 0.7411764705882353, 0.133...   
                                    molecular_function 0.35  (0.7372549019607844, 0.7411764705882353, 0.133...   
zhang_freddolino zhang_freddolino_1 biological_process 0.41  (0.596078431372549, 0.8745098039215686, 0.5411...   
                 zhang_freddolino_2 cellular_component 0.39  (0.596078431372549, 0.8745098039215686, 0.5411...   
                 zhang_freddolino_1 molecular_function 0.64  (0.596078431372549, 0.8745098039215686, 0.5411...   

                                                                  wrc  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.09  0.120647   
                                    cellular_component 0.10  0.201405   
                 DeepGO_2           molecular_function 0.41  0.135108   
INGA             INGA_1             biological_process 0.44  0.346230   
                                    cellular_component 0.55  0.569174   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.39  0.480395   
                                    molecular_function 0.35  0.384151   
zhang_freddolino zhang_freddolino_1 biological_process 0.41  0.283666   
                 zhang_freddolino_2 cellular_component 0.39  0.497686   
                 zhang_freddolino_1 molecular_function 0.64  0.352209   

                                                                  wpr  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.09  0.124663   
                                    cellular_component 0.10  0.251971   
                 DeepGO_2           molecular_function 0.41  0.137387   
INGA             INGA_1             biological_process 0.44  0.305054   
                                    cellular_component 0.55  0.449868   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.39  0.449465   
                          

In [70]:
# Set a label column for the plot legend
df_best['label'] = df_best.index.get_level_values('label')
df_best['label'] = df_best.agg(lambda x: f"{x['label']} ({metric.upper()}={x[metric]:.3f} C={x['max_cov']:.3f})", axis=1)
df_best

cov  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.09  0.851064   
                                    cellular_component 0.10  0.800940   
                 DeepGO_2           molecular_function 0.41  0.873614   
INGA             INGA_1             biological_process 0.44  0.930260   
                                    cellular_component 0.55  0.968652   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.39  0.888715   
                                    molecular_function 0.35  0.849224   
zhang_freddolino zhang_freddolino_1 biological_process 0.41  0.992908   
                 zhang_freddolino_2 cellular_component 0.39  0.995298   
                 zhang_freddolino_1 molecular_function 0.64  0.815965   

                                                                                                        colors  \
group            label              ns                 tau                                                       
DeepGO           DeepGO_1           biological_process 0.09  (0.09019607843137255, 0.7450980392156863, 0.81...   
                                    cellular_component 0.10  (0.09019607843137255, 0.7450980392156863, 0.81...   
                 DeepGO_2           molecular_function 0.41  (0.09019607843137255, 0.7450980392156863, 0.81...   
INGA             INGA_1             biological_process 0.44  (0.8392156862745098, 0.15294117647058825, 0.15...   
                                    cellular_component 0.55  (0.8392156862745098, 0.15294117647058825, 0.15...   
...                                                                                                        ...   
tianlab1         tianlab1           cellular_component 0.39  (0.7372549019607844, 0.7411764705882353, 0.133...   
                                    molecular_function 0.35  (0.7372549019607844, 0.7411764705882353, 0.133...   
zhang_freddolino zhang_freddolino_1 biological_process 0.41  (0.596078431372549, 0.8745098039215686, 0.5411...   
                 zhang_freddolino_2 cellular_component 0.39  (0.596078431372549, 0.8745098039215686, 0.5411...   
                 zhang_freddolino_1 molecular_function 0.64  (0.596078431372549, 0.8745098039215686, 0.5411...   

                                                                  wrc  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.09  0.120647   
                                    cellular_component 0.10  0.201405   
                 DeepGO_2           molecular_function 0.41  0.135108   
INGA             INGA_1             biological_process 0.44  0.346230   
                                    cellular_component 0.55  0.569174   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.39  0.480395   
                                    molecular_function 0.35  0.384151   
zhang_freddolino zhang_freddolino_1 biological_process 0.41  0.283666   
                 zhang_freddolino_2 cellular_component 0.39  0.497686   
                 zhang_freddolino_1 molecular_function 0.64  0.352209   

                                                                  wpr  \
group            label              ns                 tau              
DeepGO           DeepGO_1           biological_process 0.09  0.124663   
                                    cellular_component 0.10  0.251971   
                 DeepGO_2           molecular_function 0.41  0.137387   
INGA             INGA_1             biological_process 0.44  0.305054   
                                    cellular_component 0.55  0.449868   
...                                                               ...   
tianlab1         tianlab1           cellular_component 0.39  0.449465   
                          

In [71]:
plt.rcParams.update({'font.size': 22})
for ns, df_g in df_best.groupby(level='ns'):
    fig, ax = plt.subplots(figsize=(15, 15))
    for i, (index, row) in enumerate(df_g.sort_values(by=[metric, 'max_cov'], ascending=[False if metric in ['f', 'wf'] else True, False]).iterrows()):
        # print(row)
        data = df_methods.loc[index[:-1]]
        ax.plot(data[cols[0]], data[cols[1]], color=row['colors'], label=row['label'], lw=2, zorder=500-i)
        ax.plot(row[cols[0]], row[cols[1]], color=row['colors'], marker='o', markersize=12, mfc='none', zorder=1000-i)
        ax.plot(row[cols[0]], row[cols[1]], color=row['colors'], marker='o', markersize=6, zorder=1000-i)

    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    plt.xlim(0, max(1, df_best.loc[:,:,ns,:][cols[0]].max()))
    plt.ylim(0, max(1, df_best.loc[:,:,ns,:][cols[1]].max()))

    ax.set_title(ns)
    ax.set_xlabel(cols[0])
    ax.set_ylabel(cols[1])

    plt.savefig("{}/fig_{}_{}.png".format(out_folder, metric, ns), bbox_inches='tight')
    plt.clf()

<Figure size 1080x1080 with 0 Axes>

<Figure size 1080x1080 with 0 Axes>

<Figure size 1080x1080 with 0 Axes>